<a href="https://colab.research.google.com/github/tanasrad/Machine-Learning-for-Finance/blob/master/R_code_Machine_Learning_in_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning For Finance - Chapter 1

Klaas, J. (2019) - Machine Learning for Finance

## A logistic regressor

The simplest neural network is a logistic regressor. Logistic regression takes in values of any range but outputs only values between zero and one. There are many applications for logistic regressors. One example use case is to predict the likelihood of a homeowner to default on a mortgage. We might take all kinds of values into account to predict the likelihood of default, the debtor’s salary, whether she has a car, the security of her job, etc., but the likelihood will always be a value between zero and one. Even the worst debtor ever cannot have a default likelihood above 100% and the best cannot go below 0%.

We will use a library called numpy which enables easy and fast matrix operations in Python. To ensure we get the same result in all of our experiments, we have to set a random seed.

### Understanding regression

Recall that a line in the classical cartesian xy-plane can be defined in the form
$$
y=a+bx
$$
where $y$ indicates the dependent variable ($y$ is to be predicted) and $x$ the indipendent variable (the predictor). The slope of that line is specified by the term $b$ and indicates an increasing line if positive, and a decreasingand if negative. The intercept with the y-axis is given by the term $a$ (when $x=0$).

In machine learning a similar format of that equation is used, where the purpose of the machine is to identify values of $a$ and $b$ such that the specified line is able to describe the relationship between the supplied values $x$ to the values of $y$ in the best possible way. In practice it is rarely the case that the function perfectly relates those values, so the machine quantifies this error with an additional term. Hence, the smaller that error term, the better is the function to explain the relationship between the $y$ and $x$. 

This chapter focuses on the most basic regression models, the so-called **linear** regression models, since they use straight lines to explain the relationships. In the case of only one independent variable ($x_1$) it is called a **simple** linear regression, in the case of two or more independent variables ($x_1, x_2, ..., x_n$) it is known as **multiple** linear regression. 

## Simple linear regression

A simple linear regression model uses a line defined by an equation in the form

$$
y = \alpha + \beta x
$$

to explain the relationship between a dependent variable and a single independent variable. This equation is identical to the equation described previously.  
